딥러닝 순환망을 사용하는 생성 챗봇 개발 - LSTM

In [1]:
import numpy as np
import pandas as pd

In [2]:
with open('/content/drive/MyDrive/Colab Notebooks/13주차_과제_김정웅(20175308)/bot.txt','r') as content_file:
  botdata = content_file.read()

In [3]:
Questions = []
Answers = []

for line in botdata.split("</pattern>"):
    if "<pattern>" in line:
        Quesn = line[line.find("<pattern>")+len("<pattern>"):]
        Questions.append(Quesn.lower())
        

for line in botdata.split("</template>"):
    if "<template>" in line:
        Ans = line[line.find("<template>")+len("<template>"):]
        Ans = Ans.lower()
        Answers.append(Ans.lower())
  

QnAdata = pd.DataFrame(np.column_stack([Questions,Answers]),columns = ["Questions","Answers"])
QnAdata["QnAcomb"] = QnAdata["Questions"]+" "+QnAdata["Answers"]

print(QnAdata.head())

            Questions  ...                                            QnAcomb
0               yahoo  ...  yahoo a lot of people hear about <bot name="na...
1        you are lazy  ...       you are lazy actually i work 24 hours a day.
2         you are mad  ...    you are mad no i am quite logical and rational.
3    you are thinking  ...  you are thinking i am a thinking machine.<thin...
4  you are dividing *  ...  you are dividing * actually i am not too good ...

[5 rows x 3 columns]


In [8]:
import nltk
import collections
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
counter = collections.Counter()

for i in range(len(QnAdata)):
    for word in nltk.word_tokenize(QnAdata.iloc[i][2]):
        counter[word]+=1

word2idx = {w:(i+1) for i,(w,_) in enumerate(counter.most_common())}        
idx2word = {v:k for k,v in word2idx.items()}


idx2word[0] = "PAD"
vocab_size = len(word2idx)+1
print("Vocabulary size:",vocab_size)

Vocabulary size: 3451


In [10]:
def encode(sentence, maxlen,vocab_size):
    indices = np.zeros((maxlen, vocab_size))
    for i, w in enumerate(nltk.word_tokenize(sentence)):
        if i == maxlen: break
        indices[i, word2idx[w]] = 1
    return indices

In [11]:
def decode(indices, calc_argmax=True):
    if calc_argmax:
        indices = np.argmax(indices, axis=-1)
    return ' '.join(idx2word[x] for x in indices)

In [12]:
question_maxlen = 10
answer_maxlen = 20

def create_questions(question_maxlen,vocab_size):
    question_idx = np.zeros(shape=(len(Questions),question_maxlen,vocab_size))
    
    for q in range(len(Questions)):
        question = encode(Questions[q],question_maxlen,vocab_size)

        question_idx[i] = question 
    return question_idx

quesns_train = create_questions(question_maxlen=question_maxlen,vocab_size=vocab_size)


def create_answers(answer_maxlen,vocab_size):
    answer_idx = np.zeros(shape=(len(Answers),answer_maxlen,vocab_size))
    
    for q in range(len(Answers)):
        answer = encode(Answers[q],answer_maxlen,vocab_size)

        answer_idx[i] = answer 
    return answer_idx

answs_train = create_answers(answer_maxlen=answer_maxlen,vocab_size=vocab_size)

In [13]:
from keras.layers import Input,Dense,Dropout,Activation
from keras.models import Model
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.layers import RepeatVector,TimeDistributed,ActivityRegularization

In [14]:
n_hidden = 128

question_layer = Input(shape=(question_maxlen,vocab_size))
encoder_rnn = LSTM(n_hidden,dropout=0.2,recurrent_dropout=0.2)(question_layer)
repeat_encode = RepeatVector(answer_maxlen)(encoder_rnn)
dense_layer = TimeDistributed(Dense(vocab_size))(repeat_encode)
regularized_layer = ActivityRegularization(l2=1)(dense_layer)
softmax_layer = Activation('softmax')(regularized_layer)

model = Model([question_layer],[softmax_layer])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

print (model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10, 3451)]        0         
                                                                 
 lstm (LSTM)                 (None, 128)               1832960   
                                                                 
 repeat_vector (RepeatVector  (None, 20, 128)          0         
 )                                                               
                                                                 
 time_distributed (TimeDistr  (None, 20, 3451)         445179    
 ibuted)                                                         
                                                                 
 activity_regularization (Ac  (None, 20, 3451)         0         
 tivityRegularization)                                           
                                                             

In [15]:
quesns_train_2 = quesns_train.astype('float32')
answs_train_2 = answs_train.astype('float32')

model.fit(quesns_train_2, answs_train_2,batch_size=32,epochs=30,validation_split=0.05)

Epoch 1/30
88/88 [==============================] - 27s 258ms/step - loss: 0.0029 - accuracy: 0.0114 - val_loss: 0.0551 - val_accuracy: 0.0000e+00
Epoch 2/30
88/88 [==============================] - 22s 253ms/step - loss: 1.1101e-05 - accuracy: 0.0000e+00 - val_loss: 0.0551 - val_accuracy: 0.0000e+00
Epoch 3/30
88/88 [==============================] - 22s 254ms/step - loss: 6.6821e-06 - accuracy: 0.0000e+00 - val_loss: 0.0551 - val_accuracy: 0.0000e+00
Epoch 4/30
88/88 [==============================] - 22s 251ms/step - loss: 4.7308e-06 - accuracy: 0.0000e+00 - val_loss: 0.0551 - val_accuracy: 0.0000e+00
Epoch 5/30
88/88 [==============================] - 22s 252ms/step - loss: 3.9502e-06 - accuracy: 0.0000e+00 - val_loss: 0.0551 - val_accuracy: 0.0000e+00
Epoch 6/30
88/88 [==============================] - 22s 252ms/step - loss: 3.7584e-06 - accuracy: 0.0000e+00 - val_loss: 0.0551 - val_accuracy: 0.0000e+00
Epoch 7/30
88/88 [==============================] - 22s 251ms/step - loss: 1.1

In [19]:
ans_pred = model.predict(quesns_train_2[0:3])

print (decode(ans_pred[0]))
print (decode(ans_pred[1]))

primate primate primate primate primate primate primate primate primate primate primate primate primate primate primate primate primate primate primate primate
primate primate primate primate primate primate primate primate primate primate primate primate primate primate primate primate primate primate primate primate
